In [ ]:
import librosa
import numpy as np
from sklearn.preprocessing import minmax_scale
import os 
import matplotlib.pyplot as plt

# Neural Networks from Scratch 

Notebook originally by Louis McCallum 2022, updated by Irini Kalaitzidi 2023

Neural Networks consist of the following components:

* An input layer

* An arbitrary amount of hidden layers

* An output layer

* A set of weights and biases between each layer

* A choice of activation function for each hidden layer.
    
    
We are going to make a Neural Network that has 

* 2 Inputs (audio features)
* 1 Hidden layer (with 3 neurons)
* 1 Output (type of drum)

![fishy](../images/audio_net.png)

We have two sets of weights,  

* Input (+ bias) -> Hidden layer
    * [num_inputs + 1, num_hidden_nodes] = 3 x 3
* Hidden Layer (+ bias) -> Output
    * [num_hidden_nodes + 1, num_outputs] = 4 x 1

**Total 13 parameters**

### Randomly initiate weights

First we have to randomly initialise the weights, otherwise training will not work. We could, for example, start them all at zero, but for reasons that will become apparent, this is a bad idea. 


In [ ]:
#Determine the structure of our network (how many neurons in each layer)
#Get some data (2 inputs)
x = np.array([[3,4]])
input_size = x.shape[1]
hidden_size = 3

#Randomly initiate the weights
hidden_weights = np.random.rand(input_size + 1, hidden_size) 
output_weights = np.random.rand(hidden_size + 1, 1) 

In [ ]:
hidden_weights.shape, output_weights.shape

### Layer Activations

We also need to hold the outputs (activations) of each neuron somewhere.

For the hidden layer, there will be **3** outputs.

For the output layer, there will be **1** ouputs.

We initialise these to be zero at beginning.

In [ ]:
hidden_layer = np.zeros((hidden_size, 1))
output = np.zeros(1)

In [ ]:
hidden_layer.shape, output.shape

### Making a Forwards Pass

Now we pump some numbers into our network and get an output out the otherside. This is known as the **forwards pass**!

Each **input neuron** is connected to each **hidden layer neuron**, with a corresponding weight. We multiply each input by the weight, **sum** up all the inputs for each neuron and then pass through the **sigmoid function**.

### Matrix Multiplication 

Lets take a quick stop to remember our linear algebra. When we multiply two matrices together, we take the **dot product** of each row of the first matrix and each column of the second matrix. And the **dot product** is where we multiply two sets of numbers together then sum. Sounds familiar? 

![fishy](../images/matrix_mul.png)

### The hidden neuron activations

If the input matrix is **1 x 2**, plus a bias term **1**: ``x = [[3, 4, 1]]``
and the ``hidden_weights`` matrix is **3 x 3**, (each row representing the weights connecting a single input neuron  to each neuron in the hidden layer),
then we can multiply the two together to get a **1 x 3** matrix containing the sums of all the connections.
We then run this through the **sigmoid activation function**.

Practically speaking, we take the first row of the input ``[3,4,1]`` and the first column of the ``hidden_weights``, multiply them together **element-wise**, then **sum**. Once we have **added the bias** and passed everything through the **sigmoid function**, we get the **output** of the first hidden neuron.

```

hidden_layer[0] = sigmoid(input[0][0] * hidden_weight[0][0] + input[0][1] * hidden_weight[1][0] + hidden_weight[2][0])
hidden_layer[1] = sigmoid(input[0][0] * hidden_weight[0][1] + input[0][1] * hidden_weight[1][1] + hidden_weight[2][1])
hidden_layer[2] = sigmoid(input[0][0] * hidden_weight[0][2] + input[0][1] * hidden_weight[1][2] + hidden_weight[2][2])


```

In [ ]:
#Add one to all rows for bias term
def with_bias(x):
    x = np.array(x)
    ones = np.ones((1,x.shape[0])).T
    return np.hstack((x, ones))

In [ ]:
#Define the sigmoid function 
def sigmoid(x):
    return 1/(1 + np.exp(-x))

#Forward Pass
x = with_bias([[3,4]])
print("input \n",x)
print("hidden layer weights \n",hidden_weights)
hidden_layer = sigmoid(x @ hidden_weights)
hidden_layer = with_bias(hidden_layer)
print("hidden layer outputs \n",hidden_layer)
output = sigmoid(hidden_layer @ output_weights)
print("output layer weights \n",output_weights)
print("output \n",output)

In [ ]:
#Also works with multiple inputs at once
x = with_bias([[3,4],[5,6],[6,1]])

print("inputs \n",x)
#Forward Pass
hidden_layer = sigmoid(x @ hidden_weights)
hidden_layer = with_bias(hidden_layer)
## We get 3 sets of hidden layer outputs from 3 sets of inputs
print("hidden layer outputs \n",hidden_layer)
output = sigmoid(hidden_layer @ output_weights)
## And 3 outputs
print("output \n",output)

### The Dataset 

We are going to take some audio files of two types of drum hits: **snares** and **hihats**. 

If our model has only **2 inputs**, we need to take each audio file and get **2 features** out of them that accurately represent the similarities and differences between the two classes. 

We use the **librosa** library to get the **spectral flatness** and **spectral centroid**. These tell us something about the frequencies present in a given window in the file. 

Lets take a look at some example files:

In [ ]:
#Load audio file
hh, sr = librosa.load("../audio/drum_hits/hihat_hatsH1.WAV")
sn, sr = librosa.load("../audio/drum_hits/snare_SNR111.WAV")
#Get features for whole audio
flatness_hh = librosa.feature.spectral_flatness(y=hh,n_fft=1028)
cent_hh = librosa.feature.spectral_centroid(y=hh, sr=sr,n_fft=1028)
flatness_sn = librosa.feature.spectral_flatness(y=sn,n_fft=1028)
cent_sn = librosa.feature.spectral_centroid(y=sn, sr=sr,n_fft=1028)
fig, ax = plt.subplots(ncols=2,figsize=(12,6))
#plot
ax[0].plot(flatness_hh[0], label = "hihat")
ax[0].plot(flatness_sn[0], label = "snare")
ax[1].plot(cent_hh[0], label = "hihat")
ax[1].plot(cent_sn[0], label = "snare")
ax[0].legend()
ax[1].legend()
ax[0].set_title("Spectral Flatness")
ax[1].set_title("Spectral Centroid")

### Getting the mean 

Each feature gives us 5-6 values (depending on the length of the audio file). We only want 2 inputs per audio file!

We can see that there is some change over the file but probably the **mean** of each **feature** will be enough to differentiate between the two types of drum.

### Getting the whole set 

Below we scan the whole ``../audio/drum_hits`` folder and get the **mean spectral flatness** and **mean spectral centroid** for every file. We now have **2 features** that represent each audio file, alongside a label telling us which class they are in. 

### Normalising 

We also **normalise** each feature so they are between **0 and 1** using the ``minmax_scale`` function from the ``sklearn`` library. Before, the centroid feature went up to **7000**! Normalising data is common when training neural networks. 

In [ ]:
#Get dataset
x = []
y = []

#Walk through all the files
for root, dirs, files in os.walk("../audio/drum_hits", topdown=False):
    for name in files:
        ## Get all the ".wav" files
        if ".wav" in name.lower():
            f = os.path.join(root, name)
            #Class 0 is snares, class 1 is hihats 
            label = 0 if "snare" in name else 1
            #Load audio file
            audio, sr = librosa.load(f)
            #Get features for whole audio
            flatness = librosa.feature.spectral_flatness(y=audio,n_fft=1028)
            cent = librosa.feature.spectral_centroid(y=audio, sr=sr,n_fft=1028)
            #Get mean across the audio
            features = [np.mean(flatness), np.mean(cent)] 
            #Add to dataset
            x.append(features)
            y.append([label])

x = np.array(x)
y = np.array(y)
#Scale to between 0 and 1
x = minmax_scale(x)
y = minmax_scale(y)

### Checking out the features

We can plot our two features against each other and label the separate classes in different colours. These look like good features because the two classes look reasonably separable. 

In [ ]:
#Check out the classes and features 
fig,ax = plt.subplots()
scatter = ax.scatter(x[:,0],x[:,1],c=y)
legend1 = ax.legend(*scatter.legend_elements(), title="Drum Type")
ax.add_artist(legend1)

## Back to the Network!

We can now take all of our drums (represented by 2 features each) and run them through the network. Again, the output will be rubbish because the weights are **random**.

Here, we just pass **all the examples** all at once. We can do this because the dataset is reasonably small. When you have larger datasets, it might be necessary to pass it in **batches** (e.g. 32 at a time).

In [ ]:
#Forward Pass
hidden_layer = sigmoid(with_bias(x) @ hidden_weights)
output = sigmoid(with_bias(hidden_layer) @ output_weights)
#we get 86 outputs back!
len(output), output

### Getting good weights

We have a **Neural Network!**. But it doesn't actually do anything useful (yet). 

So, lets get some labeled data and use it to **update the weights** so that our network can give sensible predictions/labels when it sees **new data**. 

### Training 

The way we get better weights is by **training** and in the case of neural networks, training is a process of **updating the weights**. We first work out the **loss**, so that we know how good our model is. Then we are going to use this information to nudge the weights slightly, hopefully improving performance. 

If we do this enough times, we hopefully end up with weights that will transform our **input (audio features)** into an **output (type of drum)** that correctly reflects the phenomena we are trying model!

Essentially, you can think of our task as finding the **parameters** (weights) that **minimise** (get the lowest value of) the **loss function** (how bad our model is predicting labels). In this context, a low loss is a good thing!

### Gradient Descent

For that, we are going to use the **gradient descent** algorithm, an optimisation algorithm for finding the (local) minimum loss value.

![fishy](../images/grad-descent.png)

Image from https://towardsdatascience.com/how-to-build-your-own-neural-network-from-scratch-in-python-68998a08e4f6

We'll cover this in more detail in the upcoming lectures, but the basics are: 

1. Calculate the error. We will use the **mean squared error** as our **loss function**. This means that we find the difference between the output and the target label, we square it, and then find the average across all the examples.

2. For each weight in the network, determine the gradient of the loss function. This tells us the direction in which we need to head.

3. Update each weight using this gradient multiplied by a **learning rate**. Higher learning rate means more change each time. 

In [ ]:
from cci_gradients import get_gradients_bias

In [ ]:
#The Backwards Pass
#Get gradients for the mean squared error loss function (using chain rule, don't worry about this yet!)
learning_rate = 0.05
hidden_layer = sigmoid(with_bias(x) @ hidden_weights)
output = sigmoid(with_bias(hidden_layer) @ output_weights)
hidden_gradients,output_gradients = get_gradients_bias(with_bias(x), y, with_bias(hidden_layer), output, hidden_weights, output_weights, learning_rate)

#update the weights with the gradient (slope) of the loss function and learning rate
print("one gradient for each weight:")
print(hidden_weights.shape,hidden_gradients.shape)
print(output_weights.shape,output_gradients.shape)
hidden_weights = hidden_weights - (hidden_gradients * learning_rate)
output_weights = output_weights - (output_gradients * learning_rate)
#Mean Squared Error for the batch
error = np.mean(np.square(y - output))
print(error)

### Aside (Randomly Initialised Weights)

If we had initialised the weight all to zero (or any same number), then the update would be the same for every neuron! This would be the same as having **only one neuron** in each layer, which gets rid of many of the advantages of multi-layered networks.


### Batches

Couple of final things before we put it all together:

**One Epoch** is what we call one training cycle with all the data.

We can chose how much data to use at once:

* Batch Gradient Descent gives the whole dataset at once. This means we optimise the weights based on every example at once.

* Stochastic Gradient Descent uses one (randomly selected) example at a time to update the weights. 

* Mini-batch Gradient Descent takes smaller sections of the whole dataset at a time.

In the code below you can specify the batch size.


# Putting it all together 

1. Initialise the weights to random numbers

2. Split the training set into batches (anywhere from 1 -> whole dataset)

3. Forward Pass to get the outputs

4. Compare against expected outputs, find the error (loss function) and the gradient for each weight

5. Update the weights using gradients and learning rate 

6. Go back to 3 and repeat ...

In [ ]:
#Initialise
def sigmoid(x):
    return 1/(1 + np.exp(-x))

##How many hidden neurons in the hidden layer?
input_size = x.shape[1]
hidden_size = 3

hidden_weights = np.random.randn(input_size + 1, hidden_size) 
output_weights = np.random.randn(hidden_size + 1, y.shape[1])    

hidden_layer = np.zeros((hidden_size, y.shape[1]))
output = np.zeros(y.shape[1])

#Parameters (batch size, learning rate)
num_examples = x.shape[0]
batch_size = 1
batches_per_epoch = int(np.ceil(num_examples / batch_size))
epochs = 200
learning_rate = 0.1

In [ ]:
#Training
#Every epoch is every example once
for i in range(epochs):
    start = 0;
    errors = []
    for i in range(batches_per_epoch):
        
        #Get new batch of examples
        end = start + batch_size
        end = np.min([end,num_examples])
        #Batch inputs
        batch_x = x[start:end]
        #Batch output labels
        batch_y = y[start:end]

        #Forwards pass
        hidden_layer = sigmoid(with_bias(batch_x) @ hidden_weights)
        output = sigmoid(with_bias(hidden_layer) @ output_weights)
        
        #Mean Squared Error for the batch
        error = np.mean(np.square(batch_y - output))
        hidden_gradients,output_gradients = get_gradients_bias(with_bias(batch_x), batch_y, with_bias(hidden_layer), output, hidden_weights, output_weights, learning_rate)
        #update the weights with the gradient (slope) of the loss function and learning rate
        hidden_weights = hidden_weights - (hidden_gradients * learning_rate)
        output_weights = output_weights - (output_gradients * learning_rate)
        
        errors.append(error)
        start += batch_size
        
    #Print average error for each epoch
    print(np.mean(errors))
    


### Compare against non normalised features 

Why do you think it is much worse?

## Equivalent Keras Code

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [ ]:
hidden_size = 3
learning_rate = 0.1
epochs = 200
batch_size = 1
model = Sequential()
model.add(Dense(hidden_size, activation='sigmoid', input_dim=input_size))
model.add(Dense(1, activation='sigmoid'))
opt = SGD(learning_rate=learning_rate)
model.compile(opt, loss='mse',metrics=['accuracy'])
model.fit(x, y, epochs=epochs, batch_size = batch_size)

### Moving forwards

We just saw **mean squared error** loss function used with a **sigmoid** activation function on both the hidden layer and the output layer, but this is not always the right choice. 

As we move through the unit, we'll see lots of different **loss functions** and **activation functions**. However, the basic principles of optimising a neural network will remain broadly similar. 

# Lab Tasks

### Networks and Weights 

1. Given a network with 3 inputs, 1 hidden layer with 6 neurons and 2 outputs, how many parameters are there in the network in total?

2. Given a network with 4 inputs, 1 hidden layer with 6 neurons, 1 hidden layer with 3 neurons and 1 output, how many parameters are there in the network in total? 

3. If I have a network with 3 inputs and a hidden layer with 4 neurons: 

    * What is the size of my ``hidden_weights`` matrix?
    
    * When I use matrix multiplication to combine the following input dataset with the hidden weights matrix, what is the shape of the result? What do these numbers represent?
    
     ```
     inputs = [
         [2,3,4],
         [0.6,70,0.3],
         [6,5,4],
         [0.4,5,6],
     ]
     ```

### Audio Features


* The features we have picked make for quite separable data. Try the following different features and see how it updates the graph of features underneath it. Do they seem like they would be good features? Why not?

    * Standard deviation of spectral centroid and spectral flatness
    
    ```np.std(vals)```
    
    * Mean first order difference of spectral centroid and spectral flatness
     
     ```np.mean(np.diff(vals,1))) ```
    
    
* Remove the ``minmax_scaling`` from the dataset and try training the model again, are you able to get the same loss? Why do you think this might be?


### Training 

* Try experimenting with the following different parameters of the network. How does it effect the final accuracy, the speed of improvement and the amount improved each time?

    * learning rate
    
    * number of hidden neurons in the hidden layer
    
    * batch size 
    
    * number of epochs
    
    
* Make an additional array (``all errors``) and use it collect all the errors over time. Plot them in the graph using PyPlot and see how the loss decreases over time given different parameters 